In [102]:
import os
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

## Reading input files

Project root and data directory setup

In [153]:
PROJECT_ROOT = os.path.dirname(os.path.abspath(''))
DATADIR = PROJECT_ROOT + '/processed_data'

Create a list of data files and create a TF Text Dataset from the files

In [286]:
data_files = [os.path.join(DATADIR, file) for file in os.listdir(DATADIR)]
batch_size = 512
raw_data_ds = tf.data.TextLineDataset(data_files)
raw_data_ds= raw_data_ds.map(lambda x: tf.strings.split(x))
raw_data_ds=raw_data_ds.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(x))

In [309]:
def show_decoded(x):
    decoded = tf.strings.unicode_decode(x, 'utf-8').numpy()
    decoded = [chr(x) for x in decoded]
    return ''.join(decoded)

In [310]:
def show_decoded_seq(array):
    return ' '.join([show_decoded(ele) for ele in array])    

In [289]:
for elems in raw_data_ds.take(1000):
    print(show_decoded(elems.numpy()))

নিজস্ব
সংবাদদাতা
গাড়ি
নয়
যেন
মানুষের
খাঁচা
নেই
কোন
ভালো
বসার
আসন
যা
আছে
সেগুলো
ভাঙ্গাচুরা
ময়লা
ও
ধুলাবালিতে
ভরা
আবার
কখনো
ছারপোকা
কামড়ায়
তার
উপর
বসতে
গেলে
পা
গুটিয়ে
রাখতে
হয়
নেই
কোন
রুম
লাইট
জানালা
গুলোর
একটার
কাচ
থাকলে
অন্যটার
নেই
আর
যেগুলো
আছে
সেগুলোও
ভাঙ্গা
জোড়াতালি
দেয়া
ভেতরের
নোংরা
ময়লা
তো
আছেই
এবার
যাই
গাড়ির
বাহিরের
দিকে
বাহিরের
দিকে
তাকালে
প্রথমেই
নজরে
আসে
রং
উঠে
যাওয়া
দেখাচ্ছে
জং
ধরা
টিন
তার
উপরে
অসংখ্য
লম্বা
লম্বি
স্পর্ট
এ
যেন
দুমড়ে
মুচড়ে
যাওয়া
গাড়িটি
কোনভাবে
সোজা
করে
দাঁড়
করানো
হয়েছে
নেই
তাঁর
কোন
ইন্ডিকেট
কিংবা
হেডলাইট
সামনের
গ্লাসে
ও
রয়েছে
ভাঙ্গার
চিহ্ন
নগরীর
জুরাইন
থেকে
বাবুবাজার
ব্রিজ
পর্যন্ত
চলাচলকারী
বাহাদুরশাহ
পরিবহন
সম্পর্কে
বলা
হচ্ছিল
এতক্ষণ
ধরে
এখানেই
শেষ
নয়
এসব
গাড়িতে
চড়ে
যাত্রীরা
যে
আরাম
আয়েশে
থাকেন
সেটা
হয়তো
আর
ব্যাখ্যা
করার
প্রয়োজন
হবে
না
তবে
এ
পরিবহনের
সেবার
মান
না
পারলেও
বেড়েছে
ভাড়া
ও
যাত্রী
ভোগান্তি
প্রতিনিয়ত
যাত্রীদের
বিভিন্নভাবে
ভোগান্তি
পোহাতে
হচ্ছে
এ
গাড়িতে
চড়ে
বাহাদুরশাহ
পরিবহনে
নিয়মিত
চলাচল
কারী
জগন্নাথ
বিশ্ববিদ্যালয়ের
ছাত

## Generate training data

How long a sentence will be?

In [290]:
input_sequence_size = 4

Each window will have input_sequence_size + 1 words

In [291]:
sequence_data_ds=raw_data_ds.window(input_sequence_size+1, drop_remainder=True)
sequence_data_ds = sequence_data_ds.flat_map(lambda window: window.batch(5))

Split the dataset into X and y datasets

In [300]:
sequence_data_ds = sequence_data_ds.map(lambda window: (window[:-1], window[-1:]))
X_train_ds_raw = sequence_data_ds.map(lambda X,y: X)
y_train_ds_raw = sequence_data_ds.map(lambda X,y: y)


In [311]:
print("Input X  (sequence) \t\t    ----->\t Output y (next word)")
for elem1, elem2 in zip(X_train_ds_raw.take(3),y_train_ds_raw.take(3)):
   print(show_decoded_seq(elem1.numpy()),"\t\t----->\t", show_decoded_seq(elem2.numpy()))


Input X  (sequence) 		    ----->	 Output y (next word)
নিজস্ব সংবাদদাতা গাড়ি নয় 		----->	 যেন
মানুষের খাঁচা নেই কোন 		----->	 ভালো
বসার আসন যা আছে 		----->	 সেগুলো


Set the text vectorization parameters

In [243]:
max_features = 10000    # Number of distinct words in the vocabulary  
sequence_length = input_sequence_size    # Input sequence size
batch_size = 512                # Batch size

Create the text vectorization layer

In [244]:
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    # split --> DEFAULT: split each sample into substrings (usually words)
    output_mode="int",
    output_sequence_length=sequence_length,
)

Adapt the Text Vectorization layer to the training dataset

In [ ]:
vectorize_layer.adapt(raw_data_ds.take(1_000_000_000).batch(batch_size))

In [ ]:
print("The size of the vocabulary (number of distinct words): ", len(vectorize_layer.get_vocabulary()))

In [ ]:
from tensorflow.keras import layers

In [ ]:
inputs = tf.keras.Input(shape=(sequence_length), dtype="int64")
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.25)(x)
x = layers.LSTM(256, return_sequences=True)(x)
x = layers.Flatten()(x)
predictions=  layers.Dense(max_features, activation='softmax')(x)
model_LSTM = tf.keras.Model(inputs, predictions,name="model_LSTM")

In [ ]:
model_LSTM.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_LSTM.summary())

In [ ]:
model_LSTM.fit(train_ds, batch_size=256, epochs=3)

In [ ]:
model_LSTM.save(os.path.join(PROJECT_ROOT, "models"))

In [125]:
def decode_sequence(encoded_sequence):
  deceoded_sequence=[]
  for token in encoded_sequence:
    deceoded_sequence.append(vectorize_layer.get_vocabulary()[token])
  sequence= ''.join(deceoded_sequence)
  print("\t",sequence)
  return sequence